---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df = pd.read_csv('university_towns.txt', header=None, sep='\t')
    df.columns = ['combined']
    df['combined'] = df.apply(lambda x: x['combined']+'\t' if '[edit]' in x['combined'] else '\t'+x['combined'],
                              axis=1)
    df['State'] = df['combined'].str.split("\t").str[0] \
                                .str.split('[').str[0]
    df['RegionName'] = df['combined'].str.split("\t").str[1] \
                                     .str.split(' \(').str[0]
    df = df.applymap(lambda x: np.nan if (x.isspace() or x=='') else x)
    df['State'] = df['State'].fillna(method='ffill')
    df = df[df['RegionName'].notnull()].drop(['combined'], axis=1)
    
    return df

In [1]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows=7)
    df = df.iloc[:,4:7]
    df.columns = ['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars']
    df = df.set_index('Quarter')
    df = df[df.index > '2000']
    recession = []
    for i in range (np.size(df['GDP in billions of current dollars']) - 2):
        if ((df['GDP in billions of current dollars'].iloc[i+1] - df['GDP in billions of current dollars'].iloc[i]) < 0 and
            (df['GDP in billions of current dollars'].iloc[i+2] - df['GDP in billions of current dollars'].iloc[i+1]) < 0):
            recession += [df.iloc[i].name]
    return recession[0]

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls', skiprows=7)
    df = df.iloc[:,4:7]
    df.columns = ['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars']
    df = df.set_index('Quarter')
    df = df[df.index >= get_recession_start()]
    non_recession = []
    for i in range (np.size(df['GDP in billions of current dollars']) - 2):
        if ((df['GDP in billions of current dollars'].iloc[i+1] - df['GDP in billions of current dollars'].iloc[i]) > 0 and
            (df['GDP in billions of current dollars'].iloc[i+2] - df['GDP in billions of current dollars'].iloc[i+1]) > 0):
            non_recession += [df.iloc[i+2].name]
    return non_recession[0]   

In [ ]:
def get_recession_bottom():
    df = pd.read_excel('gdplev.xls', skiprows=7)
    df = df.iloc[:,4:7]
    df.columns = ['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars']
    df = df.set_index('Quarter')
    df = df[(df.index >= get_recession_start()) &
            (df.index <= get_recession_end())]
    min_gdp = np.min(df['GDP in billions of current dollars'])
    df_m_g = df[df['GDP in billions of current dollars']==min_gdp]
    return df_m_g.iloc[0].name

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('City_Zhvi_AllHomes.csv', header=0, sep=',')
    df['State'] = df['State'].map(lambda x: states[x])
    df = df.set_index(['State', 'RegionName'])
    df = df.loc[:,'2000-01':]
    df = df.T
    df.index = pd.to_datetime(df.index)
    df = df.resample('Q').mean()
    df.index = df.index.to_period("Q")
    df = df.T
    return df

In [2]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    df = convert_housing_data_to_quarters()
    recession_start = get_recession_start()
    recession_bottom = df.columns.get_loc(get_recession_bottom())
    start_m1 = df.columns.get_loc(recession_start) - 1
    ratio = df.iloc[:,start_m1] / df.iloc[:,recession_bottom]
    ratio_df = pd.DataFrame({'ratio':ratio})
    
    unis = get_list_of_university_towns()
    unis['ut'] = 1
    unis = unis.set_index(['State', 'RegionName'])
    rat_comb = pd.merge(ratio_df, unis, how='left', left_index=True, right_index=True)
    p =      ttest_ind(rat_comb[rat_comb['ut'] == 1].loc[:,'ratio'], 
                       rat_comb[rat_comb['ut'] != 1].loc[:,'ratio'], nan_policy='omit')[1]
    if p < 0.01:
        different = True
    else:
        different = False
    if np.mean(rat_comb[rat_comb['ut'] == 1].loc[:,'ratio']) > np.mean(rat_comb[rat_comb['ut'] != 1].loc[:,'ratio']):
        better = "university town"
    else:
        better = "non-university town"
        
    return (different, p, better)

NameError: name 'convert_housing_data_to_quarters' is not defined